### OCI Data Science - Useful Tips
Everything stored in the <span style="background-color: #d5d8dc ">/home/datascience</span> folder is now stored on your block volume drive. The <span style="background-color: #d5d8dc ">ads-examples</span> folder has moved outside of your working space. Notebook examples are now accessible through a Launcher tab "Notebook Examples" button.
<details>
<summary><font size="2">1. Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">2. OCI Configuration and Key Files Set Up</font></summary><p>Follow the instructions in the getting-started notebook. That notebook is accessible via the "Getting Started" Launcher tab button.</p>
</details>
<details>
<summary><font size="2">3. Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">4. Typical Cell Imports and Settings</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import MLData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">5. Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
!python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

2020-11-29 08:07:14.555070: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-29 08:07:17.004090: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-29 08:07:17.939137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-29 08:07:17.940115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2020-11-29 08:07:17.940314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NU

# Install Object Detection API

In [2]:
%%bash
mkdir TensorFlow
cd TensorFlow
git clone https://github.com/tensorflow/models

mkdir: cannot create directory ‘TensorFlow’: File exists
fatal: destination path 'models' already exists and is not an empty directory.


CalledProcessError: Command 'b'mkdir TensorFlow\ncd TensorFlow\ngit clone https://github.com/tensorflow/models\n'' returned non-zero exit status 128.

In [2]:
%%bash
cd TensorFlow/models/research
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /home/datascience/TensorFlow/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1600975 sha256=9bd420b9a97ff0bbe1214beb7c69d1816b30168cbd9eaa732bd5404f36f18e44
  Stored in directory: /tmp/pip-ephem-wheel-cache-8x3w6wmf/wheels/15/55/4c/df1a3f7681b20aab32d5bda8437d9f165ddbfe43ca45102574
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1
  Attempting uninstall: googleapis-common-protos
    Found existing installation: googleapis-common-protos 1.51.0
    Uninstalling googleapis-common-protos-1.51.0:
      Successfully uninstalled googleapis-common-protos-1.51.0


ERROR: google-api-core 1.23.0 has requirement google-auth<2.0dev,>=1.21.1, but you'll have google-auth 1.17.2 which is incompatible.


In [2]:
!python TensorFlow/models/research/object_detection/builders/model_builder_tf2_test.py

2020-11-29 08:07:39.723509: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.10: /opt/conda/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-11-29 08:07:43.617555: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-29 08:07:44.557194: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-29 08:07:44.558163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2020-11-29 08:07:44.558381: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] 

# Preparing dataset

In [6]:
%%bash
git clone https://github.com/akarazniewicz/cocosplit.git

In [4]:
%%bash
cd cocosplit
pip install -r requirements.txt

In [16]:
%%bash
python cocosplit/cocosplit.py -s 0.9976 --having-annotations export_coco/coco.json export_coco/caca.json export_coco/test_tiny.json
python cocosplit/cocosplit.py -s 0.022  --having-annotations export_coco/coco.json export_coco/train_tiny.json export_coco/caca.json

Saved 8263 entries in export_coco/caca.json and 20 in export_coco/test_tiny.json
Saved 182 entries in export_coco/train_tiny.json and 8101 in export_coco/caca.json


In [19]:
!python TensorFlow/models/research/object_detection/dataset_tools/create_coco_tf_record.py \
--train_image_dir="export_coco" \
--test_image_dir="export_coco" \
--val_image_dir="export_coco" \
--train_annotations_file="export_coco/train_tiny.json" \
--testdev_annotations_file="export_coco/test_tiny.json" \
--val_annotations_file="export_coco/test_tiny.json" \
--output_dir="tfrecord_tiny"

2020-11-29 08:31:22.616304: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
I1129 08:31:25.161962 140616274683712 create_coco_tf_record.py:403] Found groundtruth annotations. Building annotations index.
I1129 08:31:25.162433 140616274683712 create_coco_tf_record.py:416] 0 images are missing annotations.
I1129 08:31:25.163522 140616274683712 create_coco_tf_record.py:446] On image 0 of 182
I1129 08:31:26.919255 140616274683712 create_coco_tf_record.py:446] On image 100 of 182
I1129 08:31:28.432442 140616274683712 create_coco_tf_record.py:473] Finished writing, skipped 0 images.
I1129 08:31:28.432656 140616274683712 create_coco_tf_record.py:475] Skipped 3 annotations.
I1129 08:31:30.024912 140616274683712 create_coco_tf_record.py:403] Found groundtruth annotations. Building annotations index.
I1129 08:31:30.025229 140616274683712 create_coco_tf_record.py:416] 0 images are missing annotations.
I1129 08:31:30.025362 14061

# Configure training

In [5]:
%%bash
wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
tar xvfz ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
rm ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8 pre-trained-model

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


wget: /opt/conda/lib/libuuid.so.1: no version information available (required by wget)
--2020-11-29 01:26:51--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.213.16, 2a00:1450:4009:818::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.213.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 20.2M 1s
    50K .......... .......... .......... .......... ..........  0% 42.5M 1s
   100K .......... .......... .......... .......... ..........  0% 43.4M 1s
   150K .......... .......... .......... .......... ..........  0% 61.0M 1s
   200K .......... .......... .......... .......... ..........  1%  148M 0s
   250K .......... .......... .

In [40]:
%%bash
mkdir trained-model
cp pre-trained-model/pipeline.config trained-model

TODO: Modify pipeline.config

# Training

In [6]:
!cp TensorFlow/models/research/object_detection/model_main_tf2.py .

In [53]:
#%reload_ext tensorboard
#%tensorboard --logdir "/content/trained-model/train"

In [ ]:
!python model_main_tf2.py \
--model_dir=trained-model \
--pipeline_config_path=trained-model/pipeline_tiny.config \
--alsologtostderr \
--num_train_steps=50000 \
--sample_1_of_n_eval_examples=1 \
--num_eval_steps=500

2020-11-29 08:31:48.464403: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-29 08:31:52.245068: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-29 08:31:53.162274: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-29 08:31:53.163305: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2020-11-29 08:31:53.163558: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NU

# Export model

In [ ]:
!cp TensorFlow/models/research/object_detection/exporter_main_v2.py trained-model

In [9]:
%%bash
rm -rf exported-model
mkdir exported-model
cd trained-model
python exporter_main_v2.py --input_type image_tensor --pipeline_config_path pipeline.config --trained_checkpoint_dir . --output_directory ../exported-model

2020-11-29 02:16:32.070752: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-29 02:16:35.426918: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-29 02:16:36.343346: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-29 02:16:36.344318: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2020-11-29 02:16:36.344518: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NU

In [11]:
!rm saved-model.tgz
!tar -czvf saved-model.tgz exported-model/saved_model

rm: cannot remove ‘saved-model.zip’: No such file or directory
exported-model/saved_model/
exported-model/saved_model/saved_model.pb
exported-model/saved_model/assets/
exported-model/saved_model/variables/
exported-model/saved_model/variables/variables.index
exported-model/saved_model/variables/variables.data-00000-of-00001
